In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:

def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)


def run(train_index,test_index,
        dropout_rate=0.5):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  
  model=DeepConvNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate)

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =50 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
result_all= np.zeros((32,5,2))
save_file_name= 'DeepConvNetValence.npy'
for subject_id in range(32):

  Features=np.zeros((800,32,3*128))
  Labels=list()
  if subject_id % 10 ==0:
    print(subject_id)
  loaded_data=None
  file_name=None
  if subject_id<9:
      file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
  else:
      file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
  with open(file_name, 'rb') as f:
      u = pickle._Unpickler(f)
      u.encoding = 'latin1'
      loaded_data = u.load()
  data=loaded_data['data']
  Labels.append(loaded_data['labels'])

  for i in range(len(data)):
      for k in range(3,61,3):
        for j in range(32):   
          x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
          x=zscore(x)
          Features[i*20+int(k/3-1)][j]=x  


  y=np.zeros(800)
  for i in range(40):
    if Labels[0][i][0]>5:
      y[i*20:(i+1)*20]=1


  kf=KFold(n_splits=5,shuffle=True,random_state=100)

  index=0
  for train_index, test_index in kf.split(Features):
    cal=run(train_index,test_index)
    print(cal)
    result_all[subject_id,index,0]=cal[2]
    result_all[subject_id,index,1]=cal[3]

    index+=1
  
  with open(save_file_name, 'wb') as f:
      np.save(f, result_all)



Streaming output truncated to the last 5000 lines.
10/10 - 0s - loss: 0.0655 - accuracy: 0.9828 - val_loss: 0.0302 - val_accuracy: 1.0000

Epoch 00047: val_loss did not improve from 0.02993
Epoch 48/50
10/10 - 0s - loss: 0.0766 - accuracy: 0.9812 - val_loss: 0.0416 - val_accuracy: 0.9875

Epoch 00048: val_loss did not improve from 0.02993
Epoch 49/50
10/10 - 0s - loss: 0.0756 - accuracy: 0.9812 - val_loss: 0.0430 - val_accuracy: 0.9875

Epoch 00049: val_loss did not improve from 0.02993
Epoch 50/50
10/10 - 0s - loss: 0.0483 - accuracy: 0.9906 - val_loss: 0.0295 - val_accuracy: 0.9937

Epoch 00050: val_loss improved from 0.02993 to 0.02947, saving model to /tmp/checkpoint.h5
(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0,